In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
train_df=pd.read_csv('train1.csv')
test_df=pd.read_csv('test1.csv')

In [3]:
p1_dict = {
    "P1_B2004": "hx_press_sp",
    "P1_B2016": "power_press_demand",
    "P1_B3004": "rtn_level_sp",
    "P1_B3005": "rtn_flow_sp",
    "P1_B4002": "hx_temp_sp",
    "P1_B4005": "temp_pid_out",
    "P1_B400B": "heat_outflow_sp",
    "P1_B4022": "power_temp_demand",

    "P1_FCV01D": "fcv01_cmd",
    "P1_FCV01Z": "fcv01_pos",
    "P1_FCV02D": "fcv02_cmd",
    "P1_FCV02Z": "fcv02_pos",
    "P1_FCV03D": "fcv03_cmd",
    "P1_FCV03Z": "fcv03_pos",

    "P1_FT01": "rtn_flow_raw",
    "P1_FT01Z": "rtn_flow",
    "P1_FT02": "heat_flow_raw",
    "P1_FT02Z": "heat_flow",
    "P1_FT03": "rtn_flow2_raw",
    "P1_FT03Z": "rtn_flow2",

    "P1_LCV01D": "lcv01_cmd",
    "P1_LCV01Z": "lcv01_pos",

    "P1_LIT01": "rtn_level",

    "P1_PCV01D": "pcv01_cmd",
    "P1_PCV01Z": "pcv01_pos",
    "P1_PCV02D": "pcv02_cmd",
    "P1_PCV02Z": "pcv02_pos",

    "P1_PIT01": "hx_press",
    "P1_PIT01_HH": "hx_press_high",
    "P1_PIT02": "pump_supply_press",

    "P1_PP01AD": "pump1A_cmd",
    "P1_PP01AR": "pump1A_run",
    "P1_PP01BD": "pump1B_cmd",
    "P1_PP01BR": "pump1B_run",
    "P1_PP02D": "pump2_cmd",
    "P1_PP02R": "pump2_run",

    "P1_PP04": "cooler_out",
    "P1_PP04SP": "cooler_temp_sp",

    "P1_SOL01D": "sol_supply_cmd",
    "P1_SOL03D": "sol_drain_cmd",

    "P1_STSP": "boiler_run_cmd",

    "P1_TIT01": "hx_temp",
    "P1_TIT02": "heat_tank_temp",
    "P1_TIT03": "main_tank_temp"
}
train_df.rename(columns=p1_dict, inplace=True)
test_df.rename(columns=p1_dict, inplace=True)

In [4]:
''' 원본 코드
df['fcv03_diff'] = abs(df['fcv03_pos'] - df['fcv03_cmd'])
df['fcv03_cng'] = abs(df['fcv03_cmd'].diff().fillna(0))
df['level_diff'] = df['rtn_level'].diff().fillna(0)
df['level_diff_abs']=abs(df['level_diff'])
df['flow_balance'] = df['rtn_flow'] - df['rtn_flow2']
df['flow_balance_abs'] = abs(df['flow_balance'])
df['phys_diff'] = df['level_diff'] - df['flow_balance']
df['phys_diff_abs'] = abs(df['phys_diff'])
'''


" 원본 코드\ndf['fcv03_diff'] = abs(df['fcv03_pos'] - df['fcv03_cmd'])\ndf['fcv03_cng'] = abs(df['fcv03_cmd'].diff().fillna(0))\ndf['level_diff'] = df['rtn_level'].diff().fillna(0)\ndf['level_diff_abs']=abs(df['level_diff'])\ndf['flow_balance'] = df['rtn_flow'] - df['rtn_flow2']\ndf['flow_balance_abs'] = abs(df['flow_balance'])\ndf['phys_diff'] = df['level_diff'] - df['flow_balance']\ndf['phys_diff_abs'] = abs(df['phys_diff'])\n"

In [5]:
def add_custom_features(df):
    df = df.copy()

    # 1) 밸브 명령-위치 차이
    df['fcv03_diff'] = (df['fcv03_pos'] - df['fcv03_cmd']).abs()

    # 2) 밸브 명령 변화량
    df['fcv03_cng'] = df['fcv03_cmd'].diff().fillna(0).abs()

    # 3) 수위 변화량
    df['level_diff'] = df['rtn_level'].diff().fillna(0)
    df['level_diff_abs'] = df['level_diff'].abs()

    # 4) 유량 균형
    df['flow_balance'] = df['rtn_flow'] - df['rtn_flow2']
    df['flow_balance_abs'] = df['flow_balance'].abs()

    # 5) 물리 기반 차이값 (수위 변화 vs 유량 변화)
    df['phys_diff'] = df['level_diff'] - df['flow_balance']
    df['phys_diff_abs'] = df['phys_diff'].abs()

    return df


In [6]:
train_df = add_custom_features(train_df)
test_df  = add_custom_features(test_df)


In [7]:
feature_cols = [
        'phys_diff_abs',
        'flow_balance_abs',
        'level_diff_abs',
        'fcv03_cng',
        'fcv03_diff',
        'rtn_level',
        'rtn_flow',
        'rtn_flow2',
        'fcv03_cmd',
        'fcv03_pos',
        'hx_press',
        'attack'
    ]

In [8]:
train_df=train_df[feature_cols]
test_df=test_df[feature_cols]

In [9]:
feature_cols2 = [
        'phys_diff_abs',
        'flow_balance_abs',
        'level_diff_abs',
        'fcv03_cng',
        'fcv03_diff',
        'rtn_level',
        'rtn_flow',
        'rtn_flow2',
        'fcv03_cmd',
        'fcv03_pos',
        'hx_press'
    ]

In [10]:
X_train = train_df[feature_cols2]
y_train = train_df["attack"]    # all zero

X_test = test_df[feature_cols2]
y_test = test_df["attack"]


In [11]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)


In [ ]:
from xgboost import XGBClassifier

model = XGBClassifier(
    n_estimators=600,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.9,
    colsample_bytree=0.9,
    scale_pos_weight=20,   # attack이 매우 적기 때문에 가중치 필요
    eval_metric='logloss',
    random_state=42
)

model.fit(X_train_scaled, y_train)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.9, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=600, n_jobs=None,
              num_parallel_tree=None, ...)

In [16]:
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score

y_prob = model.predict_proba(X_test_scaled)[:,1]

threshold = 0.4
y_pred = (y_prob >= threshold).astype(int)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print("AUC:", roc_auc_score(y_test, y_prob))


[[42572     0]
 [  629     0]]
              precision    recall  f1-score   support

           0       0.99      1.00      0.99     42572
           1       0.00      0.00      0.00       629

    accuracy                           0.99     43201
   macro avg       0.49      0.50      0.50     43201
weighted avg       0.97      0.99      0.98     43201

AUC: 0.5


c:\Users\bora0126\AppData\Local\miniconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\bora0126\AppData\Local\miniconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\bora0126\AppData\Local\miniconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is",

# Isolation Forest

In [ ]:
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report

feature_cols = [
    'phys_diff_abs', 'flow_balance_abs', 'level_diff_abs',
    'fcv03_cng', 'fcv03_diff', 
    'rtn_level', 'rtn_flow', 'rtn_flow2',
    'fcv03_cmd', 'fcv03_pos', 'hx_press'
]

X_train = train_df[feature_cols]
X_test  = test_df[feature_cols]
y_test  = test_df['attack']   # 0/1 (test에는 공격 있음)

# 스케일링
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

# Isolation Forest (정상만 학습)
iso = IsolationForest(
    n_estimators=300,
    contamination=0.01, 
    random_state=42
)
iso.fit(X_train_scaled)

# 예측: 정상=1, 이상=-1 로 나옴
y_pred_iso = iso.predict(X_test_scaled)

# -1 → 공격(1), 1 → 정상(0) 으로 매핑
y_pred_attack = (y_pred_iso == -1).astype(int)

print(confusion_matrix(y_test, y_pred_attack))
print(classification_report(y_test, y_pred_attack))


[[42446   126]
 [  322   307]]
              precision    recall  f1-score   support

           0       0.99      1.00      0.99     42572
           1       0.71      0.49      0.58       629

    accuracy                           0.99     43201
   macro avg       0.85      0.74      0.79     43201
weighted avg       0.99      0.99      0.99     43201



In [18]:
from sklearn.metrics import recall_score
from sklearn.ensemble import IsolationForest

cont_list = [0.003, 0.005, 0.01, 0.015, 0.02]

for c in cont_list:
    iso = IsolationForest(
        n_estimators=300,
        contamination=c,
        random_state=42
    )
    iso.fit(X_train_scaled)
    
    y_pred = (iso.predict(X_test_scaled) == -1).astype(int)
    rec = recall_score(y_test, y_pred)
    print(c, rec)


0.003 0.30842607313195547
0.005 0.3863275039745628
0.01 0.48807631160572335
0.015 0.5580286168521462
0.02 0.5977742448330684


In [19]:
from sklearn.neighbors import LocalOutlierFactor

lof = LocalOutlierFactor(
    n_neighbors=50,
    novelty=True  # test에서 예측 허용
)
lof.fit(X_train_scaled)

y_pred = (lof.predict(X_test_scaled) == -1).astype(int)


In [21]:
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[31099 11473]
 [  306   323]]
              precision    recall  f1-score   support

           0       0.99      0.73      0.84     42572
           1       0.03      0.51      0.05       629

    accuracy                           0.73     43201
   macro avg       0.51      0.62      0.45     43201
weighted avg       0.98      0.73      0.83     43201



In [12]:
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report

feature_cols = [
    'phys_diff_abs', 'flow_balance_abs', 'level_diff_abs',
    'fcv03_cng', 'fcv03_diff', 
    'rtn_level', 'rtn_flow', 'rtn_flow2',
    'fcv03_cmd', 'fcv03_pos', 'hx_press'
]

X_train = train_df[feature_cols]
X_test  = test_df[feature_cols]
y_test  = test_df['attack']   # 0/1 (test에는 공격 있음)

# 스케일링
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

# Isolation Forest (정상만 학습)
iso = IsolationForest(
    n_estimators=300,
    contamination=0.02,   
    random_state=42
)
iso.fit(X_train_scaled)

# 예측: 정상=1, 이상=-1 로 나옴
y_pred_iso = iso.predict(X_test_scaled)

# -1 → 공격(1), 1 → 정상(0) 으로 매핑
y_pred_attack = (y_pred_iso == -1).astype(int)

print(confusion_matrix(y_test, y_pred_attack))
print(classification_report(y_test, y_pred_attack))

[[42322   250]
 [  253   376]]
              precision    recall  f1-score   support

           0       0.99      0.99      0.99     42572
           1       0.60      0.60      0.60       629

    accuracy                           0.99     43201
   macro avg       0.80      0.80      0.80     43201
weighted avg       0.99      0.99      0.99     43201



In [13]:
from sklearn.metrics import recall_score
from sklearn.ensemble import IsolationForest

cont_list = [0.003, 0.005,0.07, 0.01, 0.015, 0.02,0.025,0.03]

for c in cont_list:
    iso = IsolationForest(
        n_estimators=300,
        contamination=c,
        random_state=42
    )
    iso.fit(X_train_scaled)
    
    y_pred = (iso.predict(X_test_scaled) == -1).astype(int)
    rec = recall_score(y_test, y_pred)
    print(c, rec)

0.003 0.30842607313195547
0.005 0.3863275039745628
0.07 0.6677265500794912
0.01 0.48807631160572335
0.015 0.5580286168521462
0.02 0.5977742448330684
0.025 0.6200317965023847
0.03 0.6327503974562798


In [14]:
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report

feature_cols = [
    'phys_diff_abs', 'flow_balance_abs', 'level_diff_abs',
    'fcv03_cng', 'fcv03_diff', 
    'rtn_level', 'rtn_flow', 'rtn_flow2',
    'fcv03_cmd', 'fcv03_pos', 'hx_press'
]

X_train = train_df[feature_cols]
X_test  = test_df[feature_cols]
y_test  = test_df['attack']   # 0/1 (test에는 공격 있음)

# 스케일링
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

# Isolation Forest (정상만 학습)
iso = IsolationForest(
    n_estimators=300,
    contamination=0.07,   
    random_state=42
)
iso.fit(X_train_scaled)

# 예측: 정상=1, 이상=-1 로 나옴
y_pred_iso = iso.predict(X_test_scaled)

# -1 → 공격(1), 1 → 정상(0) 으로 매핑
y_pred_attack = (y_pred_iso == -1).astype(int)

print(confusion_matrix(y_test, y_pred_attack))
print(classification_report(y_test, y_pred_attack))

[[41917   655]
 [  209   420]]
              precision    recall  f1-score   support

           0       1.00      0.98      0.99     42572
           1       0.39      0.67      0.49       629

    accuracy                           0.98     43201
   macro avg       0.69      0.83      0.74     43201
weighted avg       0.99      0.98      0.98     43201



## Feature importance

- 피쳐를 망가뜨리면 얼마나 떨어지냐

In [ ]:
import numpy as np
from sklearn.metrics import f1_score
from sklearn.inspection import permutation_importance

def iso_attack_predict(model, X):
    y_pred = model.predict(X)              # 1 / -1
    return (y_pred == -1).astype(int)      # 0 / 1

def f1_scorer(estimator, X, y):
    yhat = iso_attack_predict(estimator, X)
    return f1_score(y, yhat)

result = permutation_importance(
    iso,
    X_test_scaled,
    y_test,
    scoring=f1_scorer,
    n_repeats=20,
    random_state=42,
    n_jobs=-1
)

importances = result.importances_mean
stds = result.importances_std

# 중요도 큰 순서 출력
idx = np.argsort(importances)[::-1]
for i in idx:
    print(f"{feature_cols[i]:15s}  importance={importances[i]:.4f} ± {stds[i]:.4f}")


fcv03_pos        importance=0.0178 ± 0.0015
fcv03_cmd        importance=0.0152 ± 0.0019
rtn_level        importance=0.0095 ± 0.0017
fcv03_cng        importance=0.0091 ± 0.0025
rtn_flow2        importance=0.0069 ± 0.0015
fcv03_diff       importance=0.0051 ± 0.0020
level_diff_abs   importance=0.0001 ± 0.0027
rtn_flow         importance=-0.0052 ± 0.0026
hx_press         importance=-0.0171 ± 0.0024
flow_balance_abs  importance=-0.0355 ± 0.0020
phys_diff_abs    importance=-0.0444 ± 0.0020


- tree explainer - feature의 중요도 분석/의존도

In [20]:
import shap

explainer = shap.TreeExplainer(iso)
shap_values = explainer.shap_values(X_test_scaled)

# 전역 중요도(평균 절대값)
import numpy as np
global_imp = np.mean(np.abs(shap_values), axis=0)

idx = np.argsort(global_imp)[::-1]
for i in idx:
    print(feature_cols[i], global_imp[i])


c:\Users\bora0126\AppData\Local\miniconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


level_diff_abs 0.25256525697720106
rtn_level 0.24161925839465773
rtn_flow2 0.23849502652698307
fcv03_cng 0.1873990931311346
hx_press 0.1837523392815218
fcv03_diff 0.1834999466632229
phys_diff_abs 0.16562202575707138
flow_balance_abs 0.14873116377869436
rtn_flow 0.147463197499367
fcv03_cmd 0.13134869544749303
fcv03_pos 0.10284259308066097


- 없을 때 정확도의 차이

In [21]:
from sklearn.base import clone
from sklearn.metrics import f1_score
import numpy as np

def eval_iso_f1(Xtr, Xte, yte):
    m = clone(iso)
    m.fit(Xtr)
    yhat = (m.predict(Xte) == -1).astype(int)
    return f1_score(yte, yhat)

base = eval_iso_f1(X_train_scaled, X_test_scaled, y_test)

drop_imp = []
for j, col in enumerate(feature_cols):
    Xtr_drop = np.delete(X_train_scaled, j, axis=1)
    Xte_drop = np.delete(X_test_scaled, j, axis=1)
    score = eval_iso_f1(Xtr_drop, Xte_drop, y_test)
    drop_imp.append(base - score)

idx = np.argsort(drop_imp)[::-1]
for i in idx:
    print(f"{feature_cols[i]:15s}  drop_f1={drop_imp[i]:.4f}")


rtn_flow2        drop_f1=0.0140
fcv03_pos        drop_f1=0.0128
fcv03_cmd        drop_f1=0.0100
hx_press         drop_f1=-0.0008
rtn_level        drop_f1=-0.0029
rtn_flow         drop_f1=-0.0038
fcv03_cng        drop_f1=-0.0086
fcv03_diff       drop_f1=-0.0101
flow_balance_abs  drop_f1=-0.0188
phys_diff_abs    drop_f1=-0.0210
level_diff_abs   drop_f1=-0.0307


In [13]:
y_true=y_test

### 1인 라벨에 대해서

In [22]:
from sklearn.metrics import recall_score

def attack_recall_scorer(estimator, X, y):
    y_pred = estimator.predict(X)
    y_pred_attack = (y_pred == -1).astype(int)

    return recall_score(y, y_pred_attack, pos_label=1)


In [25]:
from sklearn.base import clone
from sklearn.metrics import recall_score
import numpy as np

def eval_attack_recall(Xtr, Xte, yte):
    m = clone(iso)
    m.fit(Xtr)
    yhat = (m.predict(Xte) == -1).astype(int)
    return recall_score(yte, yhat, pos_label=1)

base_recall = eval_attack_recall(
    X_train_scaled,
    X_test_scaled,
    y_test
)

drop_imp = []
for j, col in enumerate(feature_cols):
    Xtr_drop = np.delete(X_train_scaled, j, axis=1)
    Xte_drop = np.delete(X_test_scaled, j, axis=1)
    r = eval_attack_recall(Xtr_drop, Xte_drop, y_test)
    drop_imp.append(base_recall - r)

idx = np.argsort(drop_imp)[::-1]
for i in idx:
    print(f"{feature_cols[i]:15s}  Δattack_recall={drop_imp[i]:.4f}")


fcv03_cng        Δattack_recall=0.0159
hx_press         Δattack_recall=0.0095
fcv03_diff       Δattack_recall=0.0048
rtn_level        Δattack_recall=0.0032
level_diff_abs   Δattack_recall=0.0000
flow_balance_abs  Δattack_recall=-0.0048
fcv03_pos        Δattack_recall=-0.0064
phys_diff_abs    Δattack_recall=-0.0064
fcv03_cmd        Δattack_recall=-0.0079
rtn_flow2        Δattack_recall=-0.0095
rtn_flow         Δattack_recall=-0.0111


### 재학습

In [26]:
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report

feature_cols = [
    'flow_balance_abs', 'level_diff_abs',
    'fcv03_cng', 'fcv03_diff', 
    'rtn_level', 'rtn_flow2',
    'fcv03_cmd', 'fcv03_pos', 'hx_press'
]

X_train = train_df[feature_cols]
X_test  = test_df[feature_cols]
y_test  = test_df['attack']   # 0/1 (test에는 공격 있음)

# 스케일링
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

# Isolation Forest (정상만 학습)
iso2 = IsolationForest(
    n_estimators=300,
    contamination=0.07,   
    random_state=42
)
iso2.fit(X_train_scaled)

# 예측: 정상=1, 이상=-1 로 나옴
y_pred_iso2 = iso2.predict(X_test_scaled)

# -1 → 공격(1), 1 → 정상(0) 으로 매핑
y_pred_attack2 = (y_pred_iso2 == -1).astype(int)

print(confusion_matrix(y_test, y_pred_attack2))
print(classification_report(y_test, y_pred_attack2))

[[41959   613]
 [  203   426]]
              precision    recall  f1-score   support

           0       1.00      0.99      0.99     42572
           1       0.41      0.68      0.51       629

    accuracy                           0.98     43201
   macro avg       0.70      0.83      0.75     43201
weighted avg       0.99      0.98      0.98     43201



- 성능향상

In [ ]:
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report

feature_cols = [
    'flow_balance_abs', 'level_diff_abs',
    'fcv03_cng', 'fcv03_diff', 
    'rtn_level', 'rtn_flow2',
    'fcv03_cmd', 'fcv03_pos', 'hx_press'
]

X_train = train_df[feature_cols]
X_test  = test_df[feature_cols]
y_test  = test_df['attack']   # 0/1 (test에는 공격 있음)

# 스케일링
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

# Isolation Forest (정상만 학습)
iso2 = IsolationForest(
    n_estimators=300,
    contamination=0.07,   
    random_state=42
)
iso2.fit(X_train_scaled)

# 예측: 정상=1, 이상=-1 로 나옴
y_pred_iso2 = iso2.predict(X_test_scaled)

# -1 → 공격(1), 1 → 정상(0) 으로 매핑
y_pred_attack2 = (y_pred_iso2 == -1).astype(int)

print(confusion_matrix(y_test, y_pred_attack2))
print(classification_report(y_test, y_pred_attack2))

## feature 바꿔 가면서 실험

In [27]:
from itertools import combinations
import pandas as pd
from sklearn.base import clone
from sklearn.metrics import recall_score, f1_score

base_features = [
    'flow_balance_abs', 'level_diff_abs',
    'fcv03_cng', 'fcv03_diff', 
    'rtn_level', 'rtn_flow2',
    'fcv03_cmd', 'fcv03_pos', 'hx_press'
]


In [28]:
def eval_model(features):
    Xtr = train_df[features]
    Xte = test_df[features]

    scaler = StandardScaler()
    Xtr_s = scaler.fit_transform(Xtr)
    Xte_s = scaler.transform(Xte)

    model = clone(iso)
    model.fit(Xtr_s)

    y_pred = (model.predict(Xte_s) == -1).astype(int)

    recall = recall_score(y_test, y_pred, pos_label=1)
    f1 = f1_score(y_test, y_pred)

    return recall, f1


In [29]:
results_1drop = []

for f in base_features:
    feats = [x for x in base_features if x != f]
    recall, f1 = eval_model(feats)

    results_1drop.append({
        'dropped': f,
        'used_features': feats,
        'attack_recall': recall,
        'f1': f1
    })

df_1drop = pd.DataFrame(results_1drop)
df_1drop.sort_values('attack_recall', ascending=False)


,dropped,used_features,attack_recall,f1
5,rtn_flow2,"[flow_balance_abs, level_diff_abs, fcv03_cng, ...",0.685215,0.489495
6,fcv03_cmd,"[flow_balance_abs, level_diff_abs, fcv03_cng, ...",0.680445,0.494798
7,fcv03_pos,"[flow_balance_abs, level_diff_abs, fcv03_cng, ...",0.680445,0.476615
0,flow_balance_abs,"[level_diff_abs, fcv03_cng, fcv03_diff, rtn_le...",0.678855,0.539141
3,fcv03_diff,"[flow_balance_abs, level_diff_abs, fcv03_cng, ...",0.675676,0.520196
4,rtn_level,"[flow_balance_abs, level_diff_abs, fcv03_cng, ...",0.670906,0.503881
1,level_diff_abs,"[flow_balance_abs, fcv03_cng, fcv03_diff, rtn_...",0.669316,0.564343
8,hx_press,"[flow_balance_abs, level_diff_abs, fcv03_cng, ...",0.658188,0.500605
2,fcv03_cng,"[flow_balance_abs, level_diff_abs, fcv03_diff,...",0.645469,0.533509


In [30]:
results_2drop = []

for f1, f2 in combinations(base_features, 2):
    feats = [x for x in base_features if x not in (f1, f2)]
    recall, f1_score_val = eval_model(feats)

    results_2drop.append({
        'dropped_1': f1,
        'dropped_2': f2,
        'attack_recall': recall,
        'f1': f1_score_val
    })

df_2drop = pd.DataFrame(results_2drop)
df_2drop.sort_values('attack_recall', ascending=False)


,dropped_1,dropped_2,attack_recall,f1
30,rtn_flow2,fcv03_cmd,0.689984,0.446272
31,rtn_flow2,fcv03_pos,0.688394,0.446162
5,flow_balance_abs,fcv03_cmd,0.685215,0.501454
33,fcv03_cmd,fcv03_pos,0.685215,0.470011
22,fcv03_diff,rtn_flow2,0.685215,0.472588
4,flow_balance_abs,rtn_flow2,0.683625,0.497685
2,flow_balance_abs,fcv03_diff,0.682035,0.531599
6,flow_balance_abs,fcv03_pos,0.682035,0.503817
26,rtn_level,rtn_flow2,0.682035,0.444790
24,fcv03_diff,fcv03_pos,0.682035,0.472207


In [31]:
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report

feature_cols = [
    'fcv03_cng', 'fcv03_diff', 
    'rtn_level', 'rtn_flow2',
    'fcv03_cmd', 'fcv03_pos', 'hx_press'
]

X_train = train_df[feature_cols]
X_test  = test_df[feature_cols]
y_test  = test_df['attack']   # 0/1 (test에는 공격 있음)

# 스케일링
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

# Isolation Forest (정상만 학습)
iso3 = IsolationForest(
    n_estimators=300,
    contamination=0.07,   
    random_state=42
)
iso3.fit(X_train_scaled)

# 예측: 정상=1, 이상=-1 로 나옴
y_pred_iso3 = iso3.predict(X_test_scaled)

# -1 → 공격(1), 1 → 정상(0) 으로 매핑
y_pred_attack3 = (y_pred_iso3 == -1).astype(int)

print(confusion_matrix(y_test, y_pred_attack3))
print(classification_report(y_test, y_pred_attack3))

[[42202   370]
 [  209   420]]
              precision    recall  f1-score   support

           0       1.00      0.99      0.99     42572
           1       0.53      0.67      0.59       629

    accuracy                           0.99     43201
   macro avg       0.76      0.83      0.79     43201
weighted avg       0.99      0.99      0.99     43201



## 칼럼 추가

In [34]:
extra_features = [
    "pump1A_cmd", "pump1A_run",
    "pump1B_cmd", "pump1B_run",
    "pump2_cmd",  "pump2_run",
]


In [35]:
base_features = [
    'fcv03_cng', 'fcv03_diff', 
    'rtn_level', 'rtn_flow2',
    'fcv03_cmd', 'fcv03_pos', 'hx_press'
]


In [49]:
train_df_raw=pd.read_csv('train1.csv')
test_df_raw=pd.read_csv('test1.csv')
train_df_raw.rename(columns=p1_dict, inplace=True)
test_df_raw.rename(columns=p1_dict, inplace=True)
train_df_raw = add_custom_features(train_df_raw)
test_df_raw  = add_custom_features(test_df_raw)


In [50]:
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
import numpy as np

def run_if_eval_from_raw(features, contamination=0.07):
    X_train = train_df_raw[features]
    X_test  = test_df_raw[features]
    y_test  = test_df_raw["attack"].values

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled  = scaler.transform(X_test)

    iso = IsolationForest(
        n_estimators=300,
        contamination=contamination,
        random_state=42
    )
    iso.fit(X_train_scaled)

    y_pred = (iso.predict(X_test_scaled) == -1).astype(int)

    return {
        "precision_1": precision_score(y_test, y_pred, pos_label=1, zero_division=0),
        "recall_1":    recall_score(y_test, y_pred, pos_label=1, zero_division=0),
        "f1_1":        f1_score(y_test, y_pred, pos_label=1, zero_division=0),
        "fp": int(np.sum((y_test==0) & (y_pred==1))),
        "fn": int(np.sum((y_test==1) & (y_pred==0))),
    }



In [51]:
from itertools import combinations
import pandas as pd

results = []

# (0) baseline
m = run_if_eval_from_raw(base_features)
results.append({
    "added": "(none)",
    "n_added": 0,
    "features": base_features,
    **m
})

# (1) add 1
for f in extra_features:
    feats = base_features + [f]
    m = run_if_eval_from_raw(feats)
    results.append({
        "added": f,
        "n_added": 1,
        "features": feats,
        **m
    })

# (2) add 2
for f1, f2 in combinations(extra_features, 2):
    feats = base_features + [f1, f2]
    m = run_if_eval_from_raw(feats)
    results.append({
        "added": f"{f1}+{f2}",
        "n_added": 2,
        "features": feats,
        **m
    })

df_results = pd.DataFrame(results)
df_results.sort_values(["recall_1", "f1_1"], ascending=False).head(10)



,added,n_added,features,precision_1,recall_1,f1_1,fp,fn
1,pump1A_cmd,1,"[fcv03_cng, fcv03_diff, rtn_level, rtn_flow2, ...",0.542894,0.674086,0.601418,357,205
2,pump1A_run,1,"[fcv03_cng, fcv03_diff, rtn_level, rtn_flow2, ...",0.542894,0.674086,0.601418,357,205
3,pump1B_cmd,1,"[fcv03_cng, fcv03_diff, rtn_level, rtn_flow2, ...",0.542894,0.674086,0.601418,357,205
4,pump1B_run,1,"[fcv03_cng, fcv03_diff, rtn_level, rtn_flow2, ...",0.542894,0.674086,0.601418,357,205
5,pump2_cmd,1,"[fcv03_cng, fcv03_diff, rtn_level, rtn_flow2, ...",0.542894,0.674086,0.601418,357,205
6,pump2_run,1,"[fcv03_cng, fcv03_diff, rtn_level, rtn_flow2, ...",0.542894,0.674086,0.601418,357,205
0,(none),0,"[fcv03_cng, fcv03_diff, rtn_level, rtn_flow2, ...",0.531646,0.667727,0.591966,370,209
7,pump1A_cmd+pump1A_run,2,"[fcv03_cng, fcv03_diff, rtn_level, rtn_flow2, ...",0.491587,0.650238,0.559890,423,220
8,pump1A_cmd+pump1B_cmd,2,"[fcv03_cng, fcv03_diff, rtn_level, rtn_flow2, ...",0.491587,0.650238,0.559890,423,220
9,pump1A_cmd+pump1B_run,2,"[fcv03_cng, fcv03_diff, rtn_level, rtn_flow2, ...",0.491587,0.650238,0.559890,423,220


- pump1A_cmd 추가

## 최종 모델

In [52]:
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report

# 기본 7개 + pump1A_cmd 1개 추가
feature_cols_pump1A = [
    'fcv03_cng', 'fcv03_diff',
    'rtn_level', 'rtn_flow2',
    'fcv03_cmd', 'fcv03_pos', 'hx_press',
    'pump1A_cmd'
]

# ✅ 원본 DF에서 컬럼을 다시 가져와서 조립
X_train = train_df_raw[feature_cols_pump1A]
X_test  = test_df_raw[feature_cols_pump1A]
y_test  = test_df_raw['attack']   # 0/1

# 스케일링
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

# Isolation Forest (정상만 학습)
iso_pump1A = IsolationForest(
    n_estimators=300,
    contamination=0.07,
    random_state=42
)
iso_pump1A.fit(X_train_scaled)

# 예측: 정상=1, 이상=-1
y_pred_iso = iso_pump1A.predict(X_test_scaled)

# -1 → 공격(1), 1 → 정상(0)
y_pred_attack = (y_pred_iso == -1).astype(int)

# 출력
print("Features:", feature_cols_pump1A)
print(confusion_matrix(y_test, y_pred_attack))
print(classification_report(y_test, y_pred_attack, digits=4))


Features: ['fcv03_cng', 'fcv03_diff', 'rtn_level', 'rtn_flow2', 'fcv03_cmd', 'fcv03_pos', 'hx_press', 'pump1A_cmd']
[[42215   357]
 [  205   424]]
              precision    recall  f1-score   support

           0     0.9952    0.9916    0.9934     42572
           1     0.5429    0.6741    0.6014       629

    accuracy                         0.9870     43201
   macro avg     0.7690    0.8329    0.7974     43201
weighted avg     0.9886    0.9870    0.9877     43201



In [54]:
from sklearn.base import clone
from sklearn.metrics import recall_score
import numpy as np

# pump1A_cmd 포함 feature 목록
feature_cols_pump1A = [
    'fcv03_cng', 'fcv03_diff',
    'rtn_level', 'rtn_flow2',
    'fcv03_cmd', 'fcv03_pos', 'hx_press',
    'pump1A_cmd'
]

# attack recall 평가 함수 (IsolationForest)
def eval_attack_recall(model, Xtr, Xte, yte):
    m = clone(model)
    m.fit(Xtr)
    yhat = (m.predict(Xte) == -1).astype(int)
    return recall_score(yte, yhat, pos_label=1)

# 🔹 baseline recall (모든 feature 사용)
base_recall = eval_attack_recall(
    iso_pump1A,
    X_train_scaled,
    X_test_scaled,
    y_test
)

# 🔹 drop-column importance
drop_imp = []

for j, col in enumerate(feature_cols_pump1A):
    Xtr_drop = np.delete(X_train_scaled, j, axis=1)
    Xte_drop = np.delete(X_test_scaled, j, axis=1)

    r = eval_attack_recall(
        iso_pump1A,
        Xtr_drop,
        Xte_drop,
        y_test
    )

    drop_imp.append(base_recall - r)

# 중요도 큰 순서로 정렬
idx = np.argsort(drop_imp)[::-1]

print("=== Drop-column Importance (Δ attack recall) ===")
for i in idx:
    print(f"{feature_cols_pump1A[i]:15s}  Δattack_recall={drop_imp[i]:.4f}")


=== Drop-column Importance (Δ attack recall) ===
hx_press         Δattack_recall=0.2289
fcv03_cng        Δattack_recall=0.0270
pump1A_cmd       Δattack_recall=0.0064
rtn_flow2        Δattack_recall=0.0000
rtn_level        Δattack_recall=-0.0016
fcv03_diff       Δattack_recall=-0.0064
fcv03_pos        Δattack_recall=-0.0111
fcv03_cmd        Δattack_recall=-0.0111


## OCSVM

In [ ]:
from sklearn.svm import OneClassSVM
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import recall_score, precision_score
import numpy as np

scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_test_s = scaler.transform(X_test)

svm = OneClassSVM(
    kernel="rbf",
    nu=0.03,    
    gamma='scale'
)
svm.fit(X_train_s)

scores_svm = -svm.decision_function(X_test_s)

for p in [99,98,97,95,90,85,80]:
    thr = np.percentile(scores_svm, p)
    y_pred = (scores_svm >= thr).astype(int)
    r = recall_score(y_true, y_pred)
    pr = precision_score(y_true, y_pred)
    print(f"SVM p={p:3d}  recall={r:.3f}  precision={pr:.3f}")


SVM p= 99  recall=0.569  precision=0.827
SVM p= 98  recall=0.680  precision=0.495
SVM p= 97  recall=0.690  precision=0.335
SVM p= 95  recall=0.701  precision=0.204
SVM p= 90  recall=0.727  precision=0.106
SVM p= 85  recall=0.750  precision=0.073
SVM p= 80  recall=0.766  precision=0.056


# autoencoder

In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import recall_score, precision_score, confusion_matrix, classification_report


X_train = X_train.values
X_test = X_test.values
assert isinstance(X_train, np.ndarray)
assert isinstance(X_test, np.ndarray)


X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
X_test_tensor  = torch.tensor(X_test_scaled, dtype=torch.float32)

input_dim = X_train_tensor.shape[1]


class Autoencoder(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 16),
            nn.ReLU(),
        )
        self.decoder = nn.Sequential(
            nn.Linear(16, 64),
            nn.ReLU(),
            nn.Linear(64, input_dim),
        )

    def forward(self, x):
        z = self.encoder(x)
        out = self.decoder(z)
        return out



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

model = Autoencoder(input_dim).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

batch_size = 256
train_ds = TensorDataset(X_train_tensor)  
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

n_epochs = 50
model.train()
for epoch in range(n_epochs):
    epoch_loss = 0.0
    for (x_batch,) in train_loader:
        x_batch = x_batch.to(device)
        optimizer.zero_grad()
        x_recon = model(x_batch)
        loss = criterion(x_recon, x_batch)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item() * x_batch.size(0)
    epoch_loss /= len(train_loader.dataset)
    if (epoch + 1) % 5 == 0:
        print(f"Epoch [{epoch+1}/{n_epochs}]  loss={epoch_loss:.6f}")

model.eval()
with torch.no_grad():
    X_test_tensor = X_test_tensor.to(device)
    X_test_recon  = model(X_test_tensor)
    # MSE per sample
    recon_error = torch.mean((X_test_tensor - X_test_recon) ** 2, dim=1).cpu().numpy()


y_true = y_test  # numpy 0/1


percentiles = [99, 98, 97, 95, 90, 85, 80, 75, 70]
for p in percentiles:
    thr = np.percentile(recon_error, p)
    y_pred = (recon_error >= thr).astype(int)  # 1=anomaly
    r = recall_score(y_true, y_pred)
    pr = precision_score(y_true, y_pred)
    print(f"AE p={p:3d}  thr={thr:.6f}  recall={r:.3f}  precision={pr:.3f}")


best_p = 90 
thr_best = np.percentile(recon_error, best_p)
y_pred_final = (recon_error >= thr_best).astype(int)

print("\n=== Final Threshold Evaluation (p=%d) ===" % best_p)
print("threshold:", thr_best)
print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred_final))
print("\nClassification report:")
print(classification_report(y_true, y_pred_final, digits=3))


Using device: cpu
Epoch [5/50]  loss=0.000336
Epoch [10/50]  loss=0.000095
Epoch [15/50]  loss=0.000133
Epoch [20/50]  loss=0.000139
Epoch [25/50]  loss=0.000062
Epoch [30/50]  loss=0.000055
Epoch [35/50]  loss=0.000057
Epoch [40/50]  loss=0.000088
Epoch [45/50]  loss=0.000084
Epoch [50/50]  loss=0.000076
AE p= 99  thr=0.002105  recall=0.531  precision=0.771
AE p= 98  thr=0.000491  recall=0.626  precision=0.455
AE p= 97  thr=0.000331  recall=0.649  precision=0.315
AE p= 95  thr=0.000207  recall=0.672  precision=0.196
AE p= 90  thr=0.000107  recall=0.717  precision=0.104
AE p= 85  thr=0.000074  recall=0.744  precision=0.072
AE p= 80  thr=0.000061  recall=0.765  precision=0.056
AE p= 75  thr=0.000053  recall=0.776  precision=0.045
AE p= 70  thr=0.000047  recall=0.790  precision=0.038

=== Final Threshold Evaluation (p=90) ===
threshold: 0.00010671366180758923
Confusion Matrix:
[[38702  3870]
 [  178   451]]

Classification report:
              precision    recall  f1-score   support

  

In [22]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124


Looking in indexes: https://download.pytorch.org/whl/cu124
     ---------------------------------------- 0.0/6.2 MB ? eta -:--:--
     --------------- ------------------------ 2.4/6.2 MB 11.2 MB/s eta 0:00:01
     ------------------------------ --------- 4.7/6.2 MB 11.4 MB/s eta 0:00:01
     ---------------------------------------- 6.2/6.2 MB 11.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.5 GB ? eta -:--:--
   ---------------------------------------- 0.0/2.5 GB 12.2 MB/s eta 0:03:28
   ---------------------------------------- 0.0/2.5 GB 11.9 MB/s eta 0:03:33
   ---------------------------------------- 0.0/2.5 GB 11.2 MB/s eta 0:03:46
   ---------------------------------------- 0.0/2.5 GB 10.9 MB/s eta 0:03:53
   ---------------------------------------- 0.0/2.5 GB 10.8 MB/s eta 0:03:53
   ---------------------------------------- 0.0/2.5 GB 10.5 MB/s eta 0:04:01
   ---------------------------------------- 0.0/2.5 GB 10.1 MB/s eta 0:04:09
   ----------------------

In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import recall_score, precision_score, confusion_matrix, classification_report



assert isinstance(X_train, np.ndarray)
assert isinstance(X_test, np.ndarray)


X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
X_test_tensor  = torch.tensor(X_test_scaled, dtype=torch.float32)

input_dim = X_train_tensor.shape[1]


class Autoencoder(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.encoder = nn.Sequential(
        nn.Linear(input_dim, 256),
        nn.ReLU(),
        nn.Linear(256, 128),
        nn.ReLU(),
        nn.Linear(128, 32),
        nn.ReLU(),
        )

        self.decoder = nn.Sequential(
            nn.Linear(32, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, input_dim),
        )


    def forward(self, x):
        z = self.encoder(x)
        out = self.decoder(z)
        return out



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

model = Autoencoder(input_dim).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

batch_size = 256
train_ds = TensorDataset(X_train_tensor)  
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

n_epochs = 50
model.train()
for epoch in range(n_epochs):
    epoch_loss = 0.0
    for (x_batch,) in train_loader:
        x_batch = x_batch.to(device)
        optimizer.zero_grad()
        x_recon = model(x_batch)
        loss = criterion(x_recon, x_batch)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item() * x_batch.size(0)
    epoch_loss /= len(train_loader.dataset)
    if (epoch + 1) % 5 == 0:
        print(f"Epoch [{epoch+1}/{n_epochs}]  loss={epoch_loss:.6f}")

model.eval()
with torch.no_grad():
    X_test_tensor = X_test_tensor.to(device)
    X_test_recon  = model(X_test_tensor)
    # MSE per sample
    recon_error = torch.mean((X_test_tensor - X_test_recon) ** 2, dim=1).cpu().numpy()


y_true = y_test  # numpy 0/1


percentiles = [99, 98, 97, 95, 90, 85, 80, 75, 70]
for p in percentiles:
    thr = np.percentile(recon_error, p)
    y_pred = (recon_error >= thr).astype(int)  # 1=anomaly
    r = recall_score(y_true, y_pred)
    pr = precision_score(y_true, y_pred)
    print(f"AE p={p:3d}  thr={thr:.6f}  recall={r:.3f}  precision={pr:.3f}")


best_p = 90 
thr_best = np.percentile(recon_error, best_p)
y_pred_final = (recon_error >= thr_best).astype(int)

print("\n=== Final Threshold Evaluation (p=%d) ===" % best_p)
print("threshold:", thr_best)
print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred_final))
print("\nClassification report:")
print(classification_report(y_true, y_pred_final, digits=3))

Using device: cpu
Epoch [5/50]  loss=0.001156
Epoch [10/50]  loss=0.000952
Epoch [15/50]  loss=0.000898
Epoch [20/50]  loss=0.000670
Epoch [25/50]  loss=0.000816
Epoch [30/50]  loss=0.000560
Epoch [35/50]  loss=0.000743
Epoch [40/50]  loss=0.000349
Epoch [45/50]  loss=0.000350
Epoch [50/50]  loss=0.000503
AE p= 99  thr=0.030919  recall=0.480  precision=0.697
AE p= 98  thr=0.005657  recall=0.671  precision=0.488
AE p= 97  thr=0.003715  recall=0.682  precision=0.331
AE p= 95  thr=0.002615  recall=0.695  precision=0.202
AE p= 90  thr=0.001720  recall=0.715  precision=0.104
AE p= 85  thr=0.001343  recall=0.734  precision=0.071
AE p= 80  thr=0.001103  recall=0.755  precision=0.055
AE p= 75  thr=0.000926  recall=0.765  precision=0.045
AE p= 70  thr=0.000788  recall=0.777  precision=0.038

=== Final Threshold Evaluation (p=90) ===
threshold: 0.001720063854008913
Confusion Matrix:
[[38701  3871]
 [  179   450]]

Classification report:
              precision    recall  f1-score   support

    

In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import recall_score, precision_score, confusion_matrix, classification_report



assert isinstance(X_train, np.ndarray)
assert isinstance(X_test, np.ndarray)


X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
X_test_tensor  = torch.tensor(X_test_scaled, dtype=torch.float32)

input_dim = X_train_tensor.shape[1]


class Autoencoder(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.encoder = nn.Sequential(
        nn.Linear(input_dim, 256),
        nn.ReLU(),
        nn.Linear(256, 128),
        nn.ReLU(),
        nn.Linear(128, 32),
        nn.ReLU(),
        )

        self.decoder = nn.Sequential(
            nn.Linear(32, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, input_dim),
        )


    def forward(self, x):
        z = self.encoder(x)
        out = self.decoder(z)
        return out



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

model = Autoencoder(input_dim).to(device)
criterion = nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

batch_size = 256
train_ds = TensorDataset(X_train_tensor)  
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

n_epochs = 100
model.train()
for epoch in range(n_epochs):
    epoch_loss = 0.0
    for (x_batch,) in train_loader:
        x_batch = x_batch.to(device)
        optimizer.zero_grad()
        x_recon = model(x_batch)
        loss = criterion(x_recon, x_batch)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item() * x_batch.size(0)
    epoch_loss /= len(train_loader.dataset)
    if (epoch + 1) % 5 == 0:
        print(f"Epoch [{epoch+1}/{n_epochs}]  loss={epoch_loss:.6f}")

model.eval()
with torch.no_grad():
    X_test_tensor = X_test_tensor.to(device)
    X_test_recon  = model(X_test_tensor)
    # MSE per sample
    recon_error = torch.mean((X_test_tensor - X_test_recon) ** 2, dim=1).cpu().numpy()


y_true = y_test  # numpy 0/1


percentiles = [99, 98, 97, 95, 90, 85, 80, 75, 70]
for p in percentiles:
    thr = np.percentile(recon_error, p)
    y_pred = (recon_error >= thr).astype(int)  
    r = recall_score(y_true, y_pred)
    pr = precision_score(y_true, y_pred)
    print(f"AE p={p:3d}  thr={thr:.6f}  recall={r:.3f}  precision={pr:.3f}")


best_p = 90
thr_best = np.percentile(recon_error, best_p)
y_pred_final = (recon_error >= thr_best).astype(int)

print("\n=== Final Threshold Evaluation (p=%d) ===" % best_p)
print("threshold:", thr_best)
print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred_final))
print("\nClassification report:")
print(classification_report(y_true, y_pred_final, digits=3))

Using device: cpu
Epoch [5/100]  loss=0.020449
Epoch [10/100]  loss=0.018225
Epoch [15/100]  loss=0.016121
Epoch [20/100]  loss=0.015498
Epoch [25/100]  loss=0.014960
Epoch [30/100]  loss=0.013992
Epoch [35/100]  loss=0.013873
Epoch [40/100]  loss=0.012871
Epoch [45/100]  loss=0.012392
Epoch [50/100]  loss=0.012017
Epoch [55/100]  loss=0.011889
Epoch [60/100]  loss=0.011374
Epoch [65/100]  loss=0.010901
Epoch [70/100]  loss=0.010353
Epoch [75/100]  loss=0.010446
Epoch [80/100]  loss=0.009993
Epoch [85/100]  loss=0.009876
Epoch [90/100]  loss=0.009716
Epoch [95/100]  loss=0.009634
Epoch [100/100]  loss=0.009033
AE p= 99  thr=0.002873  recall=0.541  precision=0.785
AE p= 98  thr=0.000520  recall=0.682  precision=0.496
AE p= 97  thr=0.000365  recall=0.696  precision=0.338
AE p= 95  thr=0.000275  recall=0.704  precision=0.205
AE p= 90  thr=0.000204  recall=0.723  precision=0.105
AE p= 85  thr=0.000172  recall=0.734  precision=0.071
AE p= 80  thr=0.000151  recall=0.763  precision=0.056
AE p

In [27]:
for i in range(10,100,10):
    best_p=i
    thr_best = np.percentile(recon_error, best_p)
    y_pred_final = (recon_error >= thr_best).astype(int)

    print("\n=== Final Threshold Evaluation (p=%d) ===" % best_p)
    print("threshold:", thr_best)
    print("Confusion Matrix:")
    print(confusion_matrix(y_true, y_pred_final))
    print("\nClassification report:")
    print(classification_report(y_true, y_pred_final, digits=3))


=== Final Threshold Evaluation (p=10) ===
threshold: 2.868662886612583e-05
Confusion Matrix:
[[ 4308 38264]
 [   12   617]]

Classification report:
              precision    recall  f1-score   support

           0      0.997     0.101     0.184     42572
           1      0.016     0.981     0.031       629

    accuracy                          0.114     43201
   macro avg      0.507     0.541     0.107     43201
weighted avg      0.983     0.114     0.182     43201


=== Final Threshold Evaluation (p=20) ===
threshold: 3.932914842152968e-05
Confusion Matrix:
[[ 8611 33961]
 [   29   600]]

Classification report:
              precision    recall  f1-score   support

           0      0.997     0.202     0.336     42572
           1      0.017     0.954     0.034       629

    accuracy                          0.213     43201
   macro avg      0.507     0.578     0.185     43201
weighted avg      0.982     0.213     0.332     43201


=== Final Threshold Evaluation (p=30) ===
thres